# LangChain Expression Language

- **Overview of LangChain Expression Language (LCEL)**:
  LCEL provides a declarative method for composing chains in LangChain. It's designed for ease of transitioning prototypes to production, supporting everything from simple "prompt + LLM" chains to complex ones with hundreds of steps used in production environments.

- **Key Features of LCEL**:
  - **Streaming Support**: LCEL allows for efficient streaming, minimizing time-to-first-token. This means faster and incremental outputs directly from an LLM to a streaming output parser.
  - **Async Support**: Chains built with LCEL can be executed using both synchronous (e.g., in Jupyter notebooks for prototyping) and asynchronous APIs (e.g., in LangServe servers), facilitating seamless transition from prototype to production.
  - **Optimized Parallel Execution**: LCEL automatically executes parallel steps in a chain, reducing latency in both synchronous and asynchronous modes.
  - **Retries and Fallbacks**: It offers configurable retries and fallbacks for any part of a chain, enhancing reliability at scale. Future developments include streaming support for these features to maintain low latency.
  - **Intermediate Results Access**: Useful in complex chains, this feature allows monitoring of intermediate step results for user reassurance or debugging. It's available in every LangServe server.
  - **Input and Output Schemas**: LCEL chains have Pydantic and JSONSchema schemas, inferred from the chain's structure, useful for input and output validation.
  - **Integration with LangSmith Tracing**: As chains grow in complexity, LangSmith tracing provides vital observability and debugging capabilities, with automatic logging of each step in a chain.
  - **Seamless LangServe Deployment**: Chains created with LCEL can be easily deployed using LangServe, streamlining the deployment process.

LCEL's design and features make it highly suitable for both prototyping and scaling language model applications in production environments.


## Get started

### Basic Example

- **Using the Pipe Operator (|) in LCEL**:
  - The pipe symbol (`|`) functions similarly to a Unix pipe, connecting different components.
  - It allows for the output of one component to seamlessly feed into the next, creating an integrated workflow.

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [4]:
prompt = ChatPromptTemplate.from_template(
  """
  tell me a short joke about {topic}
  """
)

model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream go to therapy? It had too many sprinkles of anxiety!'

### Prompt

- **Functionality of BasePromptTemplate**:
  - A `BasePromptTemplate` in LangChain is designed to receive a dictionary of template variables and generate a `PromptValue`.
  - The `PromptValue` is essentially a completed prompt that is adaptable for use with different types of language models.

- **Adaptability with Language Models**:
  - The `PromptValue` is versatile and can interface with either an LLM (Language Learning Model) or a ChatModel.
  - For LLMs, which require a string input, it produces a string.
  - For ChatModels, which take a sequence of messages as input, it generates `BaseMessages`.
  - This adaptability is due to the `BasePromptTemplate` having the logic to produce both `BaseMessages` and a string output.




In [5]:
prompt = ChatPromptTemplate.from_template(
  """
  tell me a short joke about {topic}
  """
)

In [6]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='\n  tell me a short joke about ice cream\n  ')])

In [7]:
prompt_value.to_messages()

[HumanMessage(content='\n  tell me a short joke about ice cream\n  ')]

In [8]:
prompt_value.to_string()

'Human: \n  tell me a short joke about ice cream\n  '

### Model

The PromptValue is then passed to model. In this case our model is a ChatModel, meaning it will output a BaseMessage.

In [9]:
model = ChatOpenAI()
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!')

If our model instead was a LLM (compared to a ChatModel), then output will be a string (instead of a BaseMessage)

In [10]:
from langchain.llms.openai import OpenAI

llm = OpenAI()
llm.invoke(prompt_value)

"\nRobot: Why don't penguins like ice cream?  Because they can't get the wrapper off."

### Output Parser

- **Function of BaseOutputParser in Model Output Processing**:
  - In LangChain, the output from a model is typically passed to a `BaseOutputParser`.
  - A `BaseOutputParser` is designed to handle different types of inputs: either a string or a `BaseMessage`.

- **Role of StrOutputParser**:
  - The `StrOutputParser`, a specific type of `BaseOutputParser`, simplifies the process by converting any input, regardless of its original form, into a string.


In [11]:
output_parser.invoke(message)

'Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!'

### Entire pipleline

- **Step-by-Step Process of Chain Execution**:
  1. **Input**: The process begins with user input in the form of a dictionary, such as `{"topic": "ice cream"}`.
  2. **Prompt Template Processing**: The input is used by the prompt component to construct a `PromptValue`. This is done by incorporating the topic into the prompt.
  3. **Model Evaluation**: The generated `PromptValue` is then fed to the `ChatModel`, specifically an OpenAI LLM, for processing. The output from the model is a `ChatMessage` object.
  4. **Output Parsing**: The `StrOutputParser` takes the `ChatMessage` and converts it into a Python string, which is the final output from the `invoke` method.

- **Component Flow**:
  - Input: `{"topic": "ice cream"}`
  - Components in sequence:
    1. `PromptTemplate`
    2. `ChatModel`
    3. `StrOutputParser`
  - Result: A Python string as the final output.

- **Testing Intermediate Results**:
  - It's possible to test smaller segments of the chain, like `prompt` or `prompt | model`, to observe the intermediate outputs and understand each component's contribution.


![](images/2023-12-17-00-07-08.png)

In [12]:
prompt = ChatPromptTemplate.from_template(
  """
  tell me a short joke about {topic}
  """
)

input = {
  "topic": "ice cream"
}

In [13]:
prompt.invoke(input)

ChatPromptValue(messages=[HumanMessage(content='\n  tell me a short joke about ice cream\n  ')])

In [14]:
(prompt | model).invoke(input)

AIMessage(content='Why did the ice cream go to therapy? Because it was feeling a little melty.')

### RAG Search Example

- **Retrieval-Augmented Generation Chain Example**:
  This example demonstrates how to set up a chain in LangChain for responding to questions with added context, using a retrieval-augmented generation approach.

- **Setup and Components**:
  - **Retriever Setup**: An in-memory store retriever is configured to fetch relevant documents based on a query, using `DocArrayInMemorySearch.from_texts`.
  - **Prompt Template**: A template is created that incorporates both the context (retrieved documents) and the user's question.
  - **Chain Components**:
    - `ChatOpenAI` (Model)
    - `StrOutputParser` (Output Parser)
    - `RunnableParallel` (Handles parallel tasks)
    - `RunnablePassthrough` (Passes the user's question)

- **Chain Composition**:
  - The chain is composed as `setup_and_retrieval | prompt | model | output_parser`.
  - `setup_and_retrieval` uses `RunnableParallel` to prepare inputs for the prompt by retrieving documents and passing the user's question.
  - The output of `setup_and_retrieval` feeds into the `prompt` to create a `PromptValue`.
  - The model processes the `PromptValue` and outputs a `ChatMessage` object.
  - The `output_parser` then converts the `ChatMessage` into a Python string.

- **Process Flow**:
  1. **Retrieval**: `retriever.invoke("where did harrison work?")` fetches relevant documents.
  2. **Input Preparation**: `RunnableParallel` combines the retrieved documents (context) and the user's question.
  3. **Prompt Generation**: User input and context are used to create a prompt.
  4. **Model Evaluation**: The prompt is evaluated by the model, generating a `ChatMessage`.
  5. **Output Parsing**: The `ChatMessage` is transformed into a string by the output parser.

- **Resulting Chain Workflow**:
  - The chain efficiently combines document retrieval with user input processing and model evaluation, culminating in a structured response.

![](images/2023-12-17-00-15-08.png)

In [18]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema.document import Document

In [16]:
vectorstore = FAISS.from_texts(
  [
    "harrison worked at kensho, bears like to eat honey"
  ],
  embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
  {
    "context": retriever,
    "question": RunnablePassthrough()
  }
)

chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")


'Harrison worked at Kensho.'

In [17]:
setup_and_retrieval.invoke("where did harrison work?")

{'context': [Document(page_content='harrison worked at kensho, bears like to eat honey')],
 'question': 'where did harrison work?'}